In [1]:
import Random
using Distributions
using Statistics
using DelimitedFiles
using Dates

In [2]:
include("Storage.jl")
include("GillespieFunctions.jl")
include("Rates.jl")

tt_protein_fb_rates! (generic function with 1 method)

In [3]:
function balance_test(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    statevector = initialstate
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    calcstor_main = Balance_with_Derivative()
    calcstor = calcstor_main.balancedata
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,calcstor,updatestorage!,rng,numsteps,maxsteps,50000)

    println("Flux Balance on x")
    println(relative_error(calcstor.meanRpX,calcstor.meanRmX))

    println("Flux Balance on y")
    println(relative_error(calcstor.meanRpY,calcstor.meanRmY))

    println("Cov Balance on x,x")
    println(cov_balance_comp(calcstor.CXRmX/weightsum,calcstor.CXRmX/weightsum,
                            calcstor.meanRpX+calcstor.meanRmX,calcstor.CXRpX/weightsum,calcstor.CXRpX/weightsum))
    println("Cov Balance on y,y")
    println(cov_balance_comp(calcstor.CYRmY/weightsum,calcstor.CYRmY/weightsum,
                            calcstor.meanRpY+calcstor.meanRmY,calcstor.CYRpY/weightsum,calcstor.CYRpY/weightsum))
    println("Cov Balance on y,x")
    println(cov_balance_comp(calcstor.CYRmX/weightsum,calcstor.CXRmY/weightsum,
                            0,calcstor.CYRpX/weightsum,calcstor.CXRpY/weightsum))
    println(steps)
    println(endstate)
    println(calcstor.meanx)
    println(calcstor.meany)


    return calcstor
end

balance_test (generic function with 1 method)

In [4]:
@time calcs = balance_test([10,50],[10.,1.,4.,1.,-1.,50.],10^2,10^10,Random.MersenneTwister())

Flux Balance on x
0.19851121789022305
Flux Balance on y
0.08717366424605855
Cov Balance on x,x
0.21667545038211228
Cov Balance on y,y
0.029608392490179197
Cov Balance on y,x
0.5740614319211529
50912
[6, 20]
5.6573828109019235
24.696150661045547
  0.167646 seconds (385.49 k allocations: 21.151 MiB, 21.91% gc time, 97.84% compilation time)


BalanceEqData(5.6573828109019235, 24.696150661045547, 103.73865627853179, 817.5481799282336, 109.1603433765791, 6.758551158574645, 5.639122152291519, 22.556488609166077, 24.710602362864268, 102.82912707449297, -8.848642241710285, 112.24973641579979, -70.44656944018918, 104.3046698768114, 411.3165082979719, 813.9320054198462, 448.99894566319915, -0.09223778506934445)

In [5]:
function data_collection(initialstate::Vector{Int64},params::Vector{Float64},numsteps::Int64,maxsteps::Int64,rng::Random.AbstractRNG)
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    datastor = BalanceEqData()
    endstate,steps,weightsum = direct_gillespie!(initialstate,params,outcomes,tt_protein_fb_rates!,datastor,updatestorage!,rng,numsteps,maxsteps,50000)
    return datastor.meanx,datastor.meany,datastor.meanF_y,datastor
end


data_collection (generic function with 1 method)

In [6]:
x,y,f_y,d = data_collection([10,50],[10.,5.,10.,1.,-3.,5.],10^7,10^10,Random.MersenneTwister())

(0.8265514523557882, 6.799167741221834, -0.9532693798944024, BalanceEqData(0.8265514523557882, 6.799167741221834, 1.9313980746099814e6, 2.3438866236714877e7, 203533.893713568, 3.5146613880701634, 3.5151885156905727, 7.030377031381145, 7.027355157067339, 7.460122789686122e6, 873636.1864150699, 6.423421321568257e6, -1.6627060288550772e7, -794158.0539330422, 1.4920245579372244e7, 2.268941703481551e7, 1.2846842643136514e7, -0.9532693798944024))

In [14]:
function CVratio(meanxs,meanys)
    return (std(meanys,corrected=false)/mean(meanys))/(std(meanxs,corrected=false)/mean(meanxs))
end
function pert_dot(initialparameters,numsamples,initialstate,numsteps,maxsteps)
    mean_xs = Vector{Float64}(undef,0)
    mean_ys = Vector{Float64}(undef,0)
    mean_fys = Vector{Float64}(undef,0)
    params = copy(initialparameters)
    outcomes = [1 0 ; -1 0; 0 1 ; 0 -1]
    statevector = copy(initialstate)
    
    for i in 1:numsamples
        statevector .= initialstate
        rng = Random.MersenneTwister()
        params[i%4+1] *= rand(Uniform(0.9, 1.1))
        datastor = BalanceEqData()
        endstate,steps,weightsum = direct_gillespie!(statevector,params,outcomes,tt_protein_fb_rates!,datastor,updatestorage!,rng,numsteps,maxsteps,50000)
        append!(mean_xs,datastor.meanx)
        append!(mean_ys,datastor.meany)
        append!(mean_fys,datastor.meanF_y)
        params[i%4+1] = initialparameters[i%4+1]
    end

    return CVratio(mean_xs,mean_ys),cor(mean_xs,mean_ys),mean(mean_fys),var(mean_xs),var(mean_ys),var(mean_fys)
end

pert_dot (generic function with 1 method)

C,rho,avgFy,varx,vary,varFy = pert_dot([40.,1.,20.,5.,0.,50.],50,[10,10],10^6,10^8)
println(join((1/C,C,rho,avgFy),' '))

In [15]:
filename = "pertresults/"*Dates.format(now(),"yyyy-udd_HHMM")*"_res.csv"
ns = [-2,-1,0,1,2]
res = Array{Float64,2}(undef,(length(ns),6))
for i in 1:length(ns)
    params = [40.,1.,20.,5.,ns[i],50.]
    res[i,:] .= pert_dot(params,50,[40,20],10^6,10^8)
end
writedlm(filename,  res, ',')

All rates trivially zero at state [0, 0] after 6110224 steps
All rates trivially zero at state [0, 0] after 20925648 steps
All rates trivially zero at state [0, 0] after 39471296 steps
All rates trivially zero at state [0, 0] after 19463370 steps
All rates trivially zero at state [0, 0] after 10090356 steps
All rates trivially zero at state [0, 0] after 3547536 steps
All rates trivially zero at state [0, 0] after 34938198 steps
All rates trivially zero at state [0, 0] after 39964080 steps
All rates trivially zero at state [0, 0] after 19610632 steps
All rates trivially zero at state [0, 0] after 21803022 steps
All rates trivially zero at state [0, 0] after 31131902 steps
All rates trivially zero at state [0, 0] after 30988322 steps
All rates trivially zero at state [0, 0] after 29407652 steps
All rates trivially zero at state [0, 0] after 3677096 steps
All rates trivially zero at state [0, 0] after 11482436 steps
All rates trivially zero at state [0, 0] after 30393296 steps
All rates t

1.5151515151515151

In [18]:
filename = "pertresults/"*Dates.format(now(),"yyyy-udd_HHMM")*"_res.csv"
ns = repeat([-2,-1,0,1,2],3)
res = Array{Float64,2}(undef,(length(ns),6))
open(filename,'w') do io
for i in 1:length(ns)
    params = [rand(Uniform(30, 50)),
                rand(Uniform(0.5, 1.5)),
                rand(Uniform(20, 30)),
                rand(Uniform(0.5,1.5)),ns[i],rand(Uniform(20,70))]
    res[i,:] .= pert_dot(params,50,[40,20],10^6,10^8)
end
writedlm(filename,  res, ',')

In [20]:
filename = "pertresults/"*Dates.format(now(),"yyyy-udd_HHMM")*"_res.csv"
ns = repeat([-3,-4,0,3,4],2)
res = Array{Float64,2}(undef,(length(ns),6))
for i in 1:length(ns)
    params = [rand(Uniform(30, 35)),
                rand(Uniform(0.5, 1.5)),
                rand(Uniform(20, 30)),
                rand(Uniform(0.5,1.5)),ns[i],rand(Uniform(30,50))]
    res[i,:] .= pert_dot(params,50,[40,20],10^6,10^8)
end
writedlm(filename,  res, ',')